In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
from IPython.display import display

In [2]:
# Define the custom R^2 metric for the Keras model
def r2_keras(output_index):
    def r2(y_true, y_pred):
        SS_res = tf.keras.backend.sum(tf.keras.backend.square(y_true[:, output_index] - y_pred[:, output_index]))
        SS_tot = tf.keras.backend.sum(tf.keras.backend.square(y_true[:, output_index] - tf.keras.backend.mean(y_true[:, output_index])))
        r2 = 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())
        return r2
    r2.__name__ = f"r2_{output_index}"
    return r2

In [3]:
# Load the test dataset
test_data = pd.read_excel('100_obs_simulation.xlsx')

# Define the inputs and output for the model
X_test = test_data[['Net Volume', 'Pulp Area', 'Froth surface area', 'Froth thickness',
       'Air Flow rate', 'R_inf Ccp', 'R_inf Gn', 'R_inf Po', 'R_inf Sp',
       'k_max Ccp', 'k_max Gn', 'k_maxPo', 'k_max Sp',
       'Entrainement Savassi parameters', 'Total Solids Flow_Feed',
       'Total Liquid Flow_Feed',
       'Pulp Volumetric Flow_Feed', 'Solids SG_Feed', 'Pulp SG_Feed',
       'Solids Fraction_Feed', 'Cu_Feed', 'Fe_Feed',
       'Pb_Feed', 'Zn_Feed']]
       
y_test = test_data[['Air Efficiency', 'Flotation Rate: Cell 1',
       'Entrainment: Cell 1', 'Total Solids Flow_Concentrate',
       'Total Liquid Flow_Concentrate',
       'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate',
       'Pulp SG_Concentrate',
       'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings',
       'Total Liquid Flow_Tailings',
       'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings',
       'Pulp SG_Tailings', 'Solids Fraction_Tailings',
       'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate',
       'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate']]

In [4]:
# Load scalers
scalerX = joblib.load('scalerX_24.joblib')
scalerY = joblib.load('scalerY_23.joblib')

# Apply the scaler to the test data
X_test = scalerX.transform(X_test)
y_test = scalerY.transform(y_test)

# Load the saved model
custom_objects = {f'r2_{i}': r2_keras(i) for i in range(23)}
model = load_model('model7_23.h5', custom_objects=custom_objects)

C:\Users\czeroo\Desktop\Managem App\SimulationApp\internship\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
# Make predictions on the preprocessed test set
y_pred = model.predict(X_test)

# Invert the scaling to get the original values
y_test = scalerY.inverse_transform(y_test)
y_pred = scalerY.inverse_transform(y_pred)

# Calculate the difference between true and predicted values
diff = np.abs(y_test - y_pred)

1/4 [======>.......................] - ETA: 1s

4/4 [==============================] - 1s 7ms/step


In [6]:
# Combine the true values, predicted values, and difference into a table
results = np.concatenate((y_test, y_pred, diff), axis=1)

columns = ['Air Efficiency', 'Flotation Rate: Cell 1',
       'Entrainment: Cell 1', 'Total Solids Flow_Concentrate',
       'Total Liquid Flow_Concentrate',
       'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate',
       'Pulp SG_Concentrate',
       'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings',
       'Total Liquid Flow_Tailings',
       'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings',
       'Pulp SG_Tailings', 'Solids Fraction_Tailings',
       'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate',
       'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate']

In [7]:
for i, col in enumerate(columns):
    output_results = results[:, [i, i + y_test.shape[1], i + 2*y_test.shape[1]]]
    output_table = pd.DataFrame(output_results, columns=['True values', 'Predicted values', 'Difference'])
    print(f"{col}")
    display(output_table)
    print(f"Average difference for {col}: {np.mean(output_table['Difference']):.2f}\n")

Air Efficiency


,True values,Predicted values,Difference
0,703.640690,686.386108,17.254581
1,721.142694,720.530029,0.612665
2,536.155348,538.245911,2.090562
3,598.180779,608.849854,10.669075
4,483.819052,485.757935,1.938882
...,...,...,...
95,1003.520646,856.355469,147.165177
96,654.565919,660.415955,5.850035
97,727.474656,716.541687,10.932969
98,677.628881,683.453308,5.824427


Average difference for Air Efficiency: 13.08

Flotation Rate: Cell 1


,True values,Predicted values,Difference
0,0.950515,0.954755,0.004240
1,0.576720,0.612979,0.036260
2,1.007466,0.978256,0.029209
3,0.605717,0.690338,0.084621
4,0.755064,0.752416,0.002648
...,...,...,...
95,0.805064,0.801360,0.003705
96,0.682673,0.633851,0.048822
97,0.675104,0.651471,0.023633
98,0.978890,0.918165,0.060724


Average difference for Flotation Rate: Cell 1: 0.03

Entrainment: Cell 1


...

Average difference for Fe_Concentrate: 0.26

Pb_Concentrate


,True values,Predicted values,Difference
0,32.558218,32.843391,0.285173
1,34.604455,34.615711,0.011256
2,34.551545,34.818130,0.266585
3,22.619529,25.760115,3.140585
4,21.132065,23.241077,2.109012
...,...,...,...
95,36.831835,38.017300,1.185465
96,38.065301,38.899845,0.834545
97,33.228366,33.736050,0.507683
98,33.043201,32.776226,0.266975


Average difference for Pb_Concentrate: 0.41

Zn_Concentrate


,True values,Predicted values,Difference
0,7.804323,7.831013,0.026691
1,7.125966,7.053392,0.072574
2,6.669077,6.712753,0.043676
3,6.982699,6.957358,0.025341
4,7.624051,7.542367,0.081684
...,...,...,...
95,6.622933,6.743792,0.120859
96,6.057328,5.831617,0.225711
97,7.204339,7.218816,0.014477
98,6.567813,6.571363,0.003550


Average difference for Zn_Concentrate: 0.15

